In [1]:
import torch
import torch.nn as nn
import argparse
from torch.utils.data import DataLoader,Sampler,Dataset,TensorDataset
from sklearn.metrics import roc_auc_score

In [2]:
# parser = argparse.ArgumentParser()
# # movie
# parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
# parser.add_argument('--n_epochs', type=int, default=20, help='the number of epochs')
# parser.add_argument('--dim', type=int, default=8, help='dimension of user and entity embeddings')
# parser.add_argument('--L', type=int, default=1, help='number of low layers')
# parser.add_argument('--H', type=int, default=1, help='number of high layers')
# parser.add_argument('--batch_size', type=int, default=4096, help='batch size')
# parser.add_argument('--l2_weight', type=float, default=1e-6, help='weight of l2 regularization')
# parser.add_argument('--lr_rs', type=float, default=0.02, help='learning rate of RS task')
# parser.add_argument('--lr_kge', type=float, default=0.01, help='learning rate of KGE task')
# parser.add_argument('--kge_interval', type=int, default=3, help='training interval of KGE task')
class args_instance:
    def __init__(self):
        self.dataset = "music"
        self.n_epochs = 20
        self.dim=8
        self.L=1
        self.H=1
        self.batch_size=4096
        self.l2_weight=1e-6
        self.lr_rs = 0.02
        self.lr_kge = 0.01
        self.kge_interval=3
args = args_instance()

In [3]:
import numpy as np
import os


def load_data(args):
    n_user, n_item, train_data, eval_data, test_data = load_rating(args)
    n_entity, n_relation, kg = load_kg(args)
    print('data loaded.')

    return n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg


def load_rating(args):
    print('reading rating file ...')

    # reading rating file
    rating_file = '../data/' + args.dataset + '/ratings_final'
    if os.path.exists(rating_file + '.npy'):
        rating_np = np.load(rating_file + '.npy')
    else:
        rating_np = np.loadtxt(rating_file + '.txt', dtype=np.int32)
        np.save(rating_file + '.npy', rating_np)

    n_user = len(set(rating_np[:, 0]))
    n_item = len(set(rating_np[:, 1]))
    train_data, eval_data, test_data = dataset_split(rating_np)

    return n_user, n_item, train_data, eval_data, test_data


def dataset_split(rating_np):
    print('splitting dataset ...')

    # train:eval:test = 6:2:2
    eval_ratio = 0.2
    test_ratio = 0.2
    n_ratings = rating_np.shape[0]

    eval_indices = np.random.choice(list(range(n_ratings)), size=int(n_ratings * eval_ratio), replace=False)
    left = set(range(n_ratings)) - set(eval_indices)
    test_indices = np.random.choice(list(left), size=int(n_ratings * test_ratio), replace=False)
    train_indices = list(left - set(test_indices))

    train_data = rating_np[train_indices]
    eval_data = rating_np[eval_indices]
    test_data = rating_np[test_indices]

    return train_data, eval_data, test_data


def load_kg(args):
    print('reading KG file ...')

    # reading kg file
    kg_file = '../data/' + args.dataset + '/kg_final'
    if os.path.exists(kg_file + '.npy'):
        kg = np.load(kg_file + '.npy')
    else:
        kg = np.loadtxt(kg_file + '.txt', dtype=np.int32)
        np.save(kg_file + '.npy', kg)

    n_entity = len(set(kg[:, 0]) | set(kg[:, 2]))
    n_relation = len(set(kg[:, 1]))

    return n_entity, n_relation, kg

data = load_data(args)

reading rating file ...
splitting dataset ...
reading KG file ...
data loaded.


In [4]:
def get_user_record(data, is_train):
    #建立每一个user交互过的item字典
    user_history_dict = dict()
    for interaction in data:
        user = interaction[0]
        item = interaction[1]
        label = interaction[2]
        if is_train or label == 1:
            if user not in user_history_dict:
                user_history_dict[user] = set()
            user_history_dict[user].add(item)
    return user_history_dict

In [5]:
show_loss = False
show_topk = False

In [6]:
#分解数据
n_user, n_item, n_entity, n_relation = data[0], data[1], data[2], data[3]
train_data, eval_data, test_data = data[4], data[5], data[6]
kg = data[7]

In [7]:
class CrossCompressUnit(nn.Module):
    def __init__(self,dim,name=None):
        super(CrossCompressUnit,self).__init__()
        self.weight_vv = nn.Parameter(nn.init.kaiming_uniform_(torch.FloatTensor(dim,1)), requires_grad=True)
        self.weight_ev = nn.Parameter(nn.init.kaiming_uniform_(torch.FloatTensor(dim,1)), requires_grad=True)
        self.weight_ve = nn.Parameter(nn.init.kaiming_uniform_(torch.FloatTensor(dim,1)), requires_grad=True)
        self.weight_ee = nn.Parameter(nn.init.kaiming_uniform_(torch.FloatTensor(dim,1)), requires_grad=True)
        
        self.bias_v = nn.Parameter(torch.FloatTensor(dim),requires_grad=True)
        self.bias_e = nn.Parameter(torch.FloatTensor(dim),requires_grad=True)
        
        self.dim = dim
    def forward(self,inputs):
        
        v,e = inputs
        # [batch_size, dim, 1], [batch_size, 1, dim]
        v = v.unsqueeze(2)
        e = e.unsqueeze(1)
        
        # [batch_size, dim, dim]
        c_matrix = v*e
        c_matrix_transpose = c_matrix.permute([0,2,1])
        
        #[batch_size * dim, dim]
        c_matrix = c_matrix.reshape([-1,self.dim])
        c_matrix_transpose = c_matrix.reshape([-1,self.dim])
        
        v_output = c_matrix.mm(self.weight_vv) + c_matrix_transpose.mm(self.weight_ev)
        v_output = v_output.reshape([-1,self.dim])
        v_output = v_output+self.bias_v
        
        e_output = c_matrix.mm(self.weight_ve) + c_matrix_transpose.mm(self.weight_ee)
        e_output = e_output.reshape([-1,self.dim])
        e_output = e_output+self.bias_e
        
        return v_output,e_output

        

In [8]:
user_num = 100
k_list = [1, 2, 5, 10, 20, 50, 100]
#获取用户交互记录
train_record = get_user_record(train_data, True)
test_record = get_user_record(test_data, False)
#全部用户列表
user_list = list(set(train_record.keys()) & set(test_record.keys()))

#取出100个用户
if len(user_list) > user_num:
    user_list = np.random.choice(user_list, size=user_num, replace=False)
#取出全部item
item_set = set(list(range(n_item)))

class MKR_RS_Dataset(Dataset):
    def __init__(self,train_dt):
        super(MKR_RS_Dataset,self).__init__()
        self.user_id = train_dt[:,0]
        self.item_id = train_dt[:,1]
        self.y = train_dt[:,2]
        self.n = train_dt.shape[0]
        
    def __getitem__(self,index):
        return [torch.tensor(self.user_id[index],dtype=torch.long),\
                torch.tensor(self.item_id[index],dtype=torch.long),\
                torch.tensor(self.y[index],dtype=torch.long)]

    def __len__(self):
        return self.n
    
mkdr_rs_ds = MKR_RS_Dataset(train_data)
mkdr_rs_dl = DataLoader(mkdr_rs_ds,batch_size=args.batch_size,shuffle=True)

class MKR_KGE_Dataset(Dataset):
    def __init__(self,kg_dt):
        super(MKR_KGE_Dataset,self).__init__()
        self.head_id = kg_dt[:,0]
        self.relation_id = kg_dt[:,1]
        self.tail_id = kg_dt[:,2]
        self.n = kg_dt.shape[0]
    
    def __getitem__(self,index):
        return [torch.tensor(self.head_id[index],dtype=torch.long),\
                torch.tensor(self.relation_id[index],dtype=torch.long),\
                torch.tensor(self.tail_id[index],dtype=torch.long)
               ]
    def __len__(self):
        return self.n
mkdr_kge_ds = MKR_KGE_Dataset(kg)
mkdr_kge_dl = DataLoader(mkdr_kge_ds,batch_size=args.batch_size,shuffle=True)


In [9]:
class MKR_torch(nn.Module):
    def __init__(self,args,n_users, n_items, n_entities, n_relations):
        super(MKR_torch, self).__init__()
        self._parse_args(n_users, n_items, n_entities, n_relations)
        self._build_model(args)
#         self._build_loss(args)
#         self._build_train(args)
        
    def _parse_args(self, n_users, n_items, n_entities, n_relations):
        self.n_user = n_users
        self.n_item = n_items
        self.n_entity = n_entities
        self.n_relation = n_relations

        # for computing l2 loss
        self.vars_rs = []
        self.vars_kge = []
    
    def forward_rs(self,user_indices,item_indices,head_indices,use_inner_product=True):
        #batch_size,dim -> [4096,8]
        self.user_embeddings = self.user_emb_matrix(user_indices)
        self.item_embeddings = self.item_emb_matrix(item_indices)
        self.head_embeddings = self.entity_emb_matrix(head_indices)

#         for i in range(args.L):
#             self.user_embeddings = self.user_mlp[i](self.user_embeddings)
#             self.item_embeddings,self.head_embeddings = self.cc_unit[i]([self.item_embeddings,self.head_embeddings])
        
        if use_inner_product:
            self.scores = torch.sum(self.user_embeddings*self.item_embeddings,axis=1)
        else:
            self.user_item_concat = torch.cat([self.user_embeddings,self.item_embeddings],axis=1)
            for i in range(args.H-1):
                self.user_item_concat=self.rs_mlps[i](self.user_item_concat)
            self.scores = self.rs_pred_mlp(self.user_item_concat).squeeze()
#         self.scores = torch.sigmoid(self.scores)
        return self.scores
    
    def forward_kge(self,item_indices,head_indices,relation_indices,tail_indices):
        self.item_embeddings = self.item_emb_matrix(item_indices)
        self.head_embeddings = self.entity_emb_matrix(head_indices)
        self.relation_embeddings = self.relation_emb_matrix(relation_indices)
        self.tail_embeddings = self.entity_emb_matrix(tail_indices)
        
        for i in range(args.L):
            self.relation_embeddings = self.rel_mlp[i](self.relation_embeddings)
            
        self.head_relation_concat = torch.cat([self.head_embeddings,self.relation_embeddings],axis=1)
        for i in range(args.H-1):
            self.head_realation_concat = self.kge_mlps[i](self.head_relation_concat)
            
        self.tail_pred = self.kge_pred_mlp(self.head_relation_concat)
        self.tail_pred = torch.sigmoid(self.tail_pred)
        #在这里直接计算损失
        self.scores_kge = torch.sum(torch.sigmoid(torch.sum(self.tail_embeddings*self.tail_pred,axis=1)))
        self.rmse=torch.mean(torch.sqrt(torch.sum((self.tail_embeddings-self.tail_pred)**2,axis=1)/args.dim))
        self.base_loss_kge = -self.scores_kge
        
        #加入l2损失
        self.l2_loss_kge = torch.sum(self.head_embeddings**2)/2 +torch.sum(self.tail_embeddings**2)/2
        self.l2_loss_kge = self.l2_loss_kge*args.l2_weight
        self.loss_kge = self.base_loss_kge + self.l2_loss_kge
#         print("loss_kge",self.loss_kge,"base",self.base_loss_kge,"l2",self.l2_loss_kge)
        return self.loss_kge,self.rmse
        
    def _build_model(self,args):
        self._build_low_layers(args)
        self._build_high_layers(args)
        
    def _build_low_layers(self,args):
        #Embedding构建
        self.user_emb_matrix = nn.Embedding(self.n_user,args.dim)
        self.item_emb_matrix = nn.Embedding(self.n_item,args.dim)
        self.entity_emb_matrix = nn.Embedding(self.n_entity,args.dim)
        self.relation_emb_matrix=  nn.Embedding(self.n_relation,args.dim)
        
        self.user_mlp=nn.ModuleList()
        self.rel_mlp=nn.ModuleList()
        self.cc_unit=nn.ModuleList()
        
        
        for _ in range(args.L):
            self.user_mlp.append(nn.Linear(args.dim,args.dim))
            self.rel_mlp.append(nn.Linear(args.dim,args.dim))
            self.cc_unit.append(CrossCompressUnit(args.dim))
            
    def _build_high_layers(self,args):
        #RS
        #拼接方法时过的mlp
        self.rs_mlps = nn.ModuleList()
        for _ in range(args.H-1):
            rs_mlp = nn.Linear(args.dim*2, args.dim*2)
            #[batch_size, dim*2]
            self.rs_mlps.append(rs_mlp)
        
        self.rs_pred_mlp = nn.Linear(args.dim*2,1)
        
        #KGE
        self.kge_mlps = nn.ModuleList()
        for _ in range(args.H-1):
            kge_mlp = nn.Linear(args.dim*2,args.dim*2)
            self.kge_mlps.append(kge_mlp)
            
        self.kge_pred_mlp = nn.Linear(args.dim*2,args.dim)
            
        

In [10]:
model = MKR_torch(args,n_user,n_item,n_entity,n_relation)

In [11]:
def eval_model(data,model):
    model.eval()
    mkdr_rs_ds = MKR_RS_Dataset(data)
    mkdr_rs_dl = DataLoader(mkdr_rs_ds,batch_size=args.batch_size,shuffle=False)
    auc_score_list = []
    auc_label_list = []
    acc_list = []
    for i, data in enumerate(mkdr_rs_dl,0):
        u_id,i_id,label = data
        scores = model.forward_rs(u_id,i_id,i_id,False)
        scores_norm = torch.sigmoid(scores).cpu().detach().numpy()
        label = label.numpy()

        predictions = [1 if i >= 0.5 else 0 for i in scores_norm]
        auc_score_list = auc_score_list + list(scores_norm)
        auc_label_list = auc_label_list + list(label)
        
        predictions = np.array(predictions)
        
#         auc = roc_auc_score(label,predictions)
#         auc_list.append(auc)
        
        acc = np.mean(np.equal(predictions,label))
        acc_list.append(acc)
    auc = roc_auc_score(auc_label_list,auc_score_list)
    return auc,np.mean(acc)

In [12]:
optimizer_rs=torch.optim.Adam(model.parameters(),lr=args.lr_rs,weight_decay=1e-6)
optimizer_kge=torch.optim.Adam(model.parameters(),lr=args.lr_kge,weight_decay=1e-6)

for step in range(100): #args.n_epochs
    start = 0
    #训练RS
    for i, data in enumerate(mkdr_rs_dl,0):
        
        u_id,i_id,label = data
        optimizer_rs.zero_grad()

        scores = model.forward_rs(u_id,i_id,i_id,use_inner_product=False) #[batch_size], 0-1的概率
        rs_loss_func = torch.nn.BCEWithLogitsLoss()
        label = torch.tensor(label,dtype=torch.float32)

        loss_rs = rs_loss_func(scores,label)
        loss_rs.backward()
        optimizer_rs.step()
        if i%10 == 0:
            print(loss_rs) 
        
    #训练KGE
    for i, data in enumerate(mkdr_kge_dl,0):
        head_id,relation_id,tail_id = data
        optimizer_kge.zero_grad()
        
        kge_loss,kge_rmse = model.forward_kge(head_id,head_id,relation_id,tail_id)
    
        kge_loss.backward()
        optimizer_kge.step()
    print(kge_loss,kge_rmse)  

    #CTR评估
    
    train_auc, train_acc = eval_model(train_data,model)
    print("train: ",train_auc,train_acc)
    eval_auc,eval_acc = eval_model(eval_data,model)
    print("eval: ",eval_auc,eval_acc)
    test_auc,test_acc = eval_model(test_data,model)
    print("test: ",test_auc,test_acc)
    
    print()
    
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor(0.7292, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(-1627.8147, grad_fn=<AddBackward0>) tensor(1.0681, grad_fn=<MeanBackward0>)
train:  0.5864700747388378 0.5721153846153846
eval:  0.5752107943200705 0.5415162454873647
test:  0.5689044958953159 0.5090252707581228

tensor(0.6824, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor(-1707.5049, grad_fn=<AddBackward0>) tensor(0.9792, grad_fn=<MeanBackward0>)
train:  0.6669893588324554 0.6213942307692307
eval:  0.6466853506452944 0.631768953068592
test:  0.6408625062168946 0.5703971119133574

tensor(0.6563, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


...

tensor(-1880.8555, grad_fn=<AddBackward0>) tensor(0.4367, grad_fn=<MeanBackward0>)
train:  0.9093169954604868 0.8269230769230769
eval:  0.7535829457104931 0.7075812274368231
test:  0.7586864196699143 0.7148014440433214

tensor(0.3872, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor(-1876.4089, grad_fn=<AddBackward0>) tensor(0.4369, grad_fn=<MeanBackward0>)
train:  0.9095257405083337 0.8269230769230769
eval:  0.7529845285502348 0.7111913357400722
test:  0.758045155149931 0.7111913357400722

tensor(0.3876, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


KeyboardInterrupt: 

In [ ]:
eval_data

In [94]:
model.entity_emb_matrix.weight

Parameter containing:
tensor([[ 0.0067,  0.3157,  2.1031,  ..., -0.4541,  0.0270,  1.0726],
        [ 0.6996,  1.1038, -1.1448,  ..., -0.4192, -0.2573, -0.1553],
        [ 1.9460, -0.5554, -0.2630,  ...,  1.5448, -0.8065, -0.0257],
        ...,
        [ 2.4704,  2.3423,  0.0296,  ..., -0.2957,  0.1612,  1.1443],
        [ 1.2935,  0.1242,  1.0175,  ...,  1.3341, -0.1081,  0.3238],
        [ 0.5405,  1.5300, -1.2162,  ...,  1.6131,  0.5859,  0.0503]],
       requires_grad=True)

In [89]:
       model.
        def forward_rs(self,user_indices,item_indices,head_indices,use_inner_product=True):
def forward_kge(self,item_indices,head_indices,realtion_indices):
    
    self,user_indices,item_indices,head_indices,relation_indices,tail_indices,user_inner_product=True
        

SyntaxError: invalid syntax (<ipython-input-89-72214e887239>, line 1)

In [60]:
tail_pred

NameError: name 'tail_pred' is not defined

In [40]:
for i in mkdr_dl:
    print(i)
    break

[tensor([5178, 2711,  182,  ..., 1163, 4618, 2118], dtype=torch.int32), tensor([ 227,   90, 2305,  ...,  930, 1766, 1294], dtype=torch.int32), tensor([1, 1, 0,  ..., 1, 0, 1], dtype=torch.int32)]


In [114]:
# tail_o = tf.random_normal([10,256])
# tail_m = tf.random_normal([10,256])

# r = tf.nn.sigmoid(tf.reduce_sum(tail_o*tail_m,axis=1))
# with tf.Session() as sess:
#     print(sess.run(r))


[1.3432661e-01 0.0000000e+00 9.9495560e-01 9.7256589e-01 5.2779913e-04
 9.6741784e-01 1.7285347e-06 7.2866678e-05 9.9999821e-01 4.9482607e-03]
